In [1]:
# Étape 1 : Chargement des bibliothèques
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib

print(" Bibliothèques chargées avec succès")

 Bibliothèques chargées avec succès


In [2]:
# Étape 2 : Chargement des données
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

print("Shape du dataset:", df.shape)
print("Colonnes disponibles:", df.columns.tolist())
print("\nAperçu des 5 premières lignes:")
df.head()

Shape du dataset: (1460, 81)
Colonnes disponibles: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Étape 3 : Sélection seulement les features numériques
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
print("Nombre total de features numériques:", len(numeric_features))

# Supprimer la target des features
if 'SalePrice' in numeric_features:
    numeric_features.remove('SalePrice')

print("Features numériques sélectionnées:", numeric_features)
print("Nombre de features utilisées:", len(numeric_features))

Nombre total de features numériques: 38
Features numériques sélectionnées: ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
Nombre de features utilisées: 37


In [4]:
# Étape 3 (suite) : Supprimer les lignes ayant des valeurs manquantes
df_numeric = df[numeric_features + ['SalePrice']].copy()
df_clean = df_numeric.dropna()

print(f"Données initiales: {df_numeric.shape}")
print(f"Données après suppression des NaN: {df_clean.shape}")
print(f"Lignes supprimées: {len(df_numeric) - len(df_clean)}")

Données initiales: (1460, 38)
Données après suppression des NaN: (1121, 38)
Lignes supprimées: 339


In [5]:
# Étape 4 : Séparation features (X) / target (y : SalePrice)
X = df_clean[numeric_features]
y = df_clean['SalePrice']

print("Features (X) shape:", X.shape)
print("Target (y) shape:", y.shape)
print("\nAperçu de la target (y):")
print(y.head())

Features (X) shape: (1121, 37)
Target (y) shape: (1121,)

Aperçu de la target (y):
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [6]:
# Étape 4 (suite) : Utiliser Stratification par binning
# Création de 5 bins pour la stratification
y_bins = pd.cut(y, bins=5, labels=False)
print("Distribution des bins:", pd.Series(y_bins).value_counts().sort_index())

Distribution des bins: SalePrice
0    659
1    386
2     65
3      7
4      4
Name: count, dtype: int64


In [7]:
# Étape 5 : Split en train / test (X_train, X_test, y_train, y_test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_bins  # Stratification pour garantir la représentativité
)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (896, 37)
X_test: (225, 37)
y_train: (896,)
y_test: (225,)


In [8]:
# Étape 6 : Entraînement du modèle (LinearRegression)
model = LinearRegression()
model.fit(X_train, y_train)

print(" Modèle LinearRegression entraîné avec succès")
print(f"Nombre de coefficients: {len(model.coef_)}")

 Modèle LinearRegression entraîné avec succès
Nombre de coefficients: 37


In [9]:
# Étape 7 : Évaluation - Scores RMSE et R2
def evaluate_model(y_true, y_pred, set_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{set_name}:")
    print(f"  - RMSE: {rmse:.2f}")
    print(f"  - R²: {r2:.4f}")
    return rmse, r2

# Prédictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(" PERFORMANCE DU MODÈLE")
print("=" * 30)
rmse_train, r2_train = evaluate_model(y_train, y_pred_train, "TRAIN")
rmse_test, r2_test = evaluate_model(y_test, y_pred_test, "TEST")

 PERFORMANCE DU MODÈLE
TRAIN:
  - RMSE: 37906.82
  - R²: 0.7914
TEST:
  - RMSE: 29400.71
  - R²: 0.8739


In [10]:
# Étape 8 : Sauvegarde du modèle model.pkl
joblib.dump(model, 'model.pkl')

# Vérification
loaded_model = joblib.load('model.pkl')
print(" Modèle sauvegardé sous 'model.pkl'")
print(" Modèle rechargé avec succès - Vérification OK")

# Lister les fichiers créés
import os
print("\n Fichiers dans le répertoire:")
for file in os.listdir('.'):
    if file.endswith('.pkl') or file.endswith('.csv'):
        print(f"  - {file}")

 Modèle sauvegardé sous 'model.pkl'
 Modèle rechargé avec succès - Vérification OK

 Fichiers dans le répertoire:
  - model.pkl


In [11]:
# VÉRIFICATION FINALE
print(" RÉSUMÉ DU PROJET")
print("=" * 40)
print(f"• Dataset: {df.shape}")
print(f"• Features numériques utilisées: {len(numeric_features)}")
print(f"• Données après nettoyage: {df_clean.shape}")
print(f"• Split train/test: {X_train.shape[0]}/{X_test.shape[0]}")
print(f"• Performance TEST - RMSE: {rmse_test:.2f}, R²: {r2_test:.4f}")
print(f"• Modèle sauvegardé: model.pkl")

# print("\n ACTIVITÉ 0 TERMINÉE AVEC SUCCÈS!")
# print(" Prêt pour l'Activité 1: Versionnement avec Git/GitHub")

 RÉSUMÉ DU PROJET
• Dataset: (1460, 81)
• Features numériques utilisées: 37
• Données après nettoyage: (1121, 38)
• Split train/test: 896/225
• Performance TEST - RMSE: 29400.71, R²: 0.8739
• Modèle sauvegardé: model.pkl


In [12]:
from IPython.display import FileLink

# Télécharger le notebook
FileLink('house_prices_poc.ipynb')

# Télécharger le modèle
FileLink('model.pkl')

# Télécharger le requirements.txt (à créer)
with open('requirements.txt', 'w') as f:
    f.write('''pandas==1.5.3
numpy==1.24.3
scikit-learn==1.2.2
matplotlib==3.7.1
jupyter==1.0.0
joblib==1.2.0
seaborn==0.12.2''')

FileLink('requirements.txt')

/kaggle/working/requirements.txt